<a href="https://colab.research.google.com/github/arbi11/CompEM1/blob/master/A2C_tf2_CNN_BC_Lives_DyNA_3_Dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

M_NO                = 1
MACHINE             = 'Acer'
#PATH = "D:\\Documents\\Codes\\RL-Acer\\RL\\PPO"
PATH                = 'C:\\Users\\chetanm\\Desktop\\Deep Learning\\A2C_tf2_CNN_stack'

MODEL_PATH          = 'Ampere_Data_Eval_DQL_1N3_A2C_NL_STACK_5_33_lives'
TRAIN_PATH          = 'Ampere_Data_Train_DQL_1N3_A2C_NL'
max_steps           = 75
env_dim             = [18, 35]
action_dim          = 8
state_size          = [4, 15, 28]
max_iron            = 190

lr                  = 0.0007
batch_size          = 1024
miniBatchSize       = 512
penalty             = -2

gamma               = 0.95                             # Discounting rate
playTime            = 2

warmUpSteps         = 500 #200
experienceLength    = 2500
hidden_size         = [512, 256]
variance            = 0.4
num_episodes        = 500

max_eps             = 1.0
min_eps             = 0.01
decay_rate          = 0.001
 
tau                 = 0.001
entropy             = 0.001
value_multiplier    = 0.5


In [0]:
import numpy as np
import femm
import imageio
import warnings
from  skimage import img_as_float, img_as_uint
#import matplotlib.pyplot as plt
import re
import os
import constants as con
from shutil import copyfile
import pickle

FEMM_PATH = con.PATH + '\\'
#FEMM_PATH = 'C:\\Users\\Arbi\\Desktop\\Arbi\\'
MODEL_PATH = con.MODEL_PATH + '\\'
TRAIN_PATH = con.TRAIN_PATH + '\\'

class WormFemmEnv4:

    def __init__(self, env_dim = [21, 35], startR=3 , startC= 4, max_steps= con.max_steps):
        
        self.env_dim = env_dim
        self.step_count = 0
        self.posR = startR
        self.posC = startC
        self.done = 0.0
        self.repeat = 0
        self.issue = 0
        self.max_steps = max_steps
        self.past_rewards = []
        self.past_states = []
        self.past_actions = []
        self.state_hist = []
        self.past_BnStates = []
        self.net_force = 0.0
        self.R = 0.0
        self.count = 0
        self.buffer = 3
        self.B_state = np.zeros([15,])
        
        self.coil_start = 2
        self.coil_width = 3
        self.coil_length = 9
        
        self.coil_current_choice = np.array([1, 1.5])
        self.coil_turns_choice = np.array([500])
        
        self.arm_start = 27
        self.arm_width = 6
        self.arm_length = 15
        
        self.cont_start_col = self.coil_start + self.coil_width
        self.window_width = self.arm_start - (self.coil_start + self.coil_width)
        self.window_length = np.max([self.arm_length, 15])
        self.frame_idx = 0
        self.worm_step_size = 3
        
        self.lives = 0.75
        self.material_left = (con.max_iron - 9)/con.max_iron
        self.steps_left = (con.max_steps - 1)/con.max_steps
        
    
    def create_geo(self):
        
        femm.openfemm(1);
        femm.opendocument(FEMM_PATH + 'actuator.fem');

        femm.mi_getmaterial('Air')
#        femm.mi_addmaterial('LinearIron', 2100, 2100, 0, 0, 0, 0, 0, 1, 0, 0, 0)
        femm.mi_getmaterial('Cold rolled low carbon strip steel')            
        femm.mi_getmaterial('Copper')
        femm.mi_addcircprop('icoil', 1, 1);
        
        # femm.mi_setblockprop('Air', 0, 10, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        for i in range(17):
            for j in range(35):
                
                femm.mi_selectlabel(i + 0.5, j + 0.5);
                
        femm.mi_setblockprop('Air', 0, 0.5, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        coil2_start = 8 + 8 - (self.coil_start + self.coil_width)
        femm.mi_clearselected();
        
        femm.mi_selectrectangle(0 + 0.1, self.coil_start + 0.1, self.coil_length - 0.1, self.coil_start + self.coil_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()
        femm.mi_drawrectangle(0, self.coil_start, self.coil_length, self.coil_start + self.coil_width)
        femm.mi_addblocklabel((0 + self.coil_length)/ 2, (2*self.coil_start + self.coil_width)/ 2)
        
        femm.mi_clearselected();

        femm.mi_selectrectangle(0 + 0.1, coil2_start + 0.1, self.coil_length - 0.1, coil2_start + self.coil_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()        
        femm.mi_drawrectangle(0, coil2_start, self.coil_length, coil2_start + self.coil_width)
        femm.mi_addblocklabel((0 + self.coil_length)/ 2, (2*coil2_start + self.coil_width)/ 2)
        femm.mi_clearselected();

        femm.mi_selectlabel((0 + self.coil_length)/ 2, (2*coil2_start + self.coil_width)/ 2);
        femm.mi_setblockprop('Copper', 0, 0, 'icoil', 0, 1, 500); 
        femm.mi_clearselected();

        femm.mi_selectlabel((0 + self.coil_length)/ 2, (2*self.coil_start + self.coil_width)/ 2)
        femm.mi_setblockprop('Copper', 0, 0, 'icoil', 0, 1, -500); 
        
        #        femm.mi_setblockprop('Copper', 0, 0.25, 'icoil', 0, 1, 500);        
        femm.mi_clearselected();

        femm.mi_selectrectangle(0 + 0.1, self.arm_start + 0.1, self.arm_length - 0.1, self.arm_start + self.arm_width - 0.1, 4)
        femm.mi_deleteselectedlabels()
        femm.mi_deleteselectednodes()
        femm.mi_deleteselectedsegments()        
        femm.mi_drawrectangle(0, self.arm_start, self.arm_length, self.arm_start + self.arm_width)
        femm.mi_addblocklabel((0 + self.arm_length)/ 2, (2*self.arm_start + self.arm_width)/ 2)
        femm.mi_selectlabel((0 + self.arm_length)/ 2, (2*self.arm_start + self.arm_width)/ 2);
        femm.mi_setblockprop('Cold rolled low carbon strip steel', 0, 0.5, '<None>', 0, 5, 0);
        femm.mi_clearselected();
                
        femm.mi_drawrectangle(0, 100, 100, -100)
        femm.mi_addblocklabel(50, 0);
        femm.mi_selectlabel(50, 0);
        femm.mi_setblockprop('Air', 50, 0, '<None>', 0, 0, 0);
        femm.mi_clearselected();

        femm.mi_addboundprop('neumann', 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0)
        femm.mi_selectrectangle(-0.5, 100.5, 0.2, -100.5, 4)
        femm.mi_setsegmentprop('neumann', 0, 0, 0, 5)
        femm.mi_clearselected();       
        
        femm.mi_selectsegment(95, -95)
        femm.mi_selectsegment(55, 95)
        femm.mi_selectsegment(55, -95)
        femm.mi_addboundprop('dirichlet', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
        femm.mi_setsegmentprop('dirichlet', 0, 0, 0, 0)
        femm.mi_clearselected();       

        femm.mi_zoomnatural();
        femm.mi_saveas(FEMM_PATH + 'actuator2.fem');  
        femm.closefemm()
        
        #####################################################################################
        #                                   VARIABLES                                       #       
        #####################################################################################
        
        # step_count           :       step# in an episode.
        # done          :       episode over or not.
        # snap          :       store material dist as png (mat2gray).
        # act_py        :       action (material dist) sent to matlab.
        # state_abs     :       action with discrete values(0/1) & flattened.
        # changes       :       changes made between (n-1) & 'n'th actions.
        # act_mat       :       action as matlab double.
        # count         :       No. of iron (1's) in the action.
        # frame_idx     :       episode #.
        
        #####################################################################################
        # mat_res       :       output from matlab:                                         #
        #                               nargout= 2 (means 2 variables from matlab)          #
        # field_xy      :                   var1 = Bx, By (concatenated) dim: (2, 21, 15)   #
        # Fx            :                   var2 = Force magnitude dim: 1(scalar)           #
        #####################################################################################
        
        # reward        :       Reward for the agent (Fx(n) - Fx(n-1)).
        # state3        :       environment state for the agent. dim: (3, 21, 15).
        
        #####################################################################################
        
    def move(self, mat_state):
        
        if (self.worm_step_size == 3):
            mat_state[self.posR -1 : self.posR + 2, self.posC -1 : self.posC + 2] = 4    
            mat_state[self.posR , self.posC] = 5
        elif (self.worm_step_size == 1 and self.past_actions[-1]<4):
            mat_state[self.posR -1 : self.posR + 2, self.posC -1 : self.posC + 2] = 4    
            mat_state[self.posR , self.posC] = 5
        else:
            mat_state[self.posR , self.posC] = 5

        return(mat_state)

    def reward(self, mat_state):
        
        #print('entered REWARD process with a shape of:', mat_state.shape)

        femm.openfemm(1);
        femm.opendocument(FEMM_PATH + 'actuator2.fem');
        #femm.mi_addmaterial('LinearIron', 2100, 2100, 0, 0, 0, 0, 0, 1, 0, 0, 0)

        indices = np.nonzero(mat_state[self.buffer:-1, :] - self.state_hist[-1])
        
        len_ind = np.size(indices[0])
        for i in range(len_ind):
            #print(indices[0][i], ',', indices[1][i])
            femm.mi_selectlabel(indices[0][i] + 0.4, indices[1][i] + 0.4);
            femm.mi_deleteselectedlabels()
            femm.mi_addblocklabel(indices[0][i] + 0.4, indices[1][i] + 0.4)
            femm.mi_clearselected();

        for i in range(len_ind):
            femm.mi_selectlabel(indices[0][i] + 0.4, indices[1][i] + 0.4);
            femm.mi_setblockprop('Cold rolled low carbon strip steel', 0, 0, '<None>', 0, 3, 0);
            
        femm.mi_clearselected();
        femm.mi_zoomnatural()
        femm.mi_saveas('actuator2.fem')
        femm.mi_analyse(0)
        femm.mi_loadsolution()
                
        Bx, By = femm.mo_getb(0.2, 27.12)
        B = np.sqrt(Bx ** 2 + By ** 2)
        femm.mo_groupselectblock(5)
        self.net_force = femm.mo_blockintegral(19) *(-1)
                  
        Bxy_state = np.zeros([self.env_dim[0]-self.buffer, self.env_dim[1], 2])
        for r in range(Bxy_state.shape[0]):
            for c in range(self.env_dim[1]):
                Bxy_state[r, c, :] = femm.mo_getb(r+0.5, c+0.5)
            
        self.B_state = np.sum(np.square(Bxy_state), axis=-1)                
        femm.closefemm()        
        reward = B*1000
                
        return(reward)            
        
    def check_actual_movement(self, clean_state):
        clean_state [clean_state ==5] = 4
        clean_past_state = np.copy(self.past_states[-1])
        clean_past_state [clean_past_state == 5] = 4
        
        if np.array_equal(clean_state, clean_past_state):
            #print('same as before')
            return True            
        else:
            return False
    
    def clean_up(self, mat_state):
        
        mat_state[self.buffer: self.buffer + self.coil_length, 
                  self.coil_start: self.coil_start + self.coil_width] = -self.coil_current
        coil2_start = 8 + 8 - (self.coil_start + self.coil_width)
        mat_state[:, 0:self.coil_start] = -3
        mat_state[self.buffer: self.buffer + self.coil_length, 
                  coil2_start : coil2_start + self.coil_width] = -self.coil_current
                  
        mat_state[self.buffer + self.coil_length:, 
                  self.coil_start: self.coil_start + self.coil_width] = -3                  
        mat_state[0:self.buffer, :] = -3
        mat_state[:, self.arm_start- 1] = -3
        mat_state[self.window_length + self.buffer:, :] = -3
        mat_state[self.buffer: self.buffer + self.arm_length, 
                  self.arm_start : self.arm_start + self.arm_width] = -4
                  
        mat_state[self.buffer + self.coil_length:, self.coil_start+1] = np.copy(self.lives)
        mat_state[self.buffer + self.coil_length:, self.coil_start] = np.copy(self.material_left)
        mat_state[self.buffer + self.coil_length:, self.coil_start+2] = np.copy(self.steps_left)

                          
        mat_state[mat_state == 5] = 4
        mat_state[self.posR, self.posC] = 5

        return mat_state
    
    ##### ACTIONS #######
    # 0     : soft RIGHT#
    # 1     : soft LEFT #
    # 2     : soft UP   #
    # 3     : soft DOWN #
    ##-----------------##
    # 4     : RIGHT     #
    # 5     : LEFT      #
    # 6     : UP        #
    # 7     : DOWN      #
    #####################
    
    def step(self, action):
        
        state = np.copy(self.past_states[-1])
        self.step_count +=1
        self.R = 0.0
        self.count = 0

        if action == 0:
            self.worm_step_size = 1
            self.posC += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.issue = 2
                self.posC -= self.worm_step_size
                        
        elif action == 1:
            self.worm_step_size = 1                               
            self.posC -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posC += self.worm_step_size
                self.issue = 2
        
        elif action == 2:
            self.worm_step_size = 1                               
            self.posR -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posR += self.worm_step_size
                self.issue = 2
        
        elif action == 3:
            self.worm_step_size = 1    
            self.posR += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                state2 = self.move(state)
                self.issue = 1
            else:
                self.posR -= self.worm_step_size
                self.issue = 2
        
        elif action == 4:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3  
                
            self.posC += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.issue = 2
                self.posC -= self.worm_step_size
                        
        elif action == 5:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3                    
            
            self.posC -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posC += self.worm_step_size
                self.issue = 2
        
        elif action == 6:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3
                
            self.posR -= self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                self.issue = 1
                state2 = self.move(state)
            else:
                self.posR += self.worm_step_size
                self.issue = 2
        
        elif action == 7:
            if self.past_actions[-1]<4:
                self.worm_step_size = 1
            else:
                self.worm_step_size = 3
            self.posR += self.worm_step_size
            
            if state[self.posR, self.posC] == 0:    
                state2 = self.move(state)
                self.issue = 0
            elif state[self.posR, self.posC] == 4:
                state2 = self.move(state)
                self.issue = 1
            else:
                self.posR -= self.worm_step_size
                self.issue = 2
                
        if self.issue == 0:    
            #print('Going for reward with a shape of:', state2.shape)
            new_state = self.clean_up(state2)
            F = self.reward(new_state)
            self.R = F - np.copy(self.past_forces[-1])
#            R = F
            self.past_rewards.append(self.R)
            self.past_forces.append(np.copy(F))
           
        elif self.issue == 1:
                                 
            new_state = self.clean_up(state2)
            if not self.check_actual_movement(np.copy(new_state)):
                
                #print('Not the same geo. Going for reward')
                F = self.reward(new_state)
                self.R = F - np.copy(self.past_forces[-1])           
                self.past_rewards.append(self.R)
                self.past_forces.append(np.copy(F))
            else:
                self.R = 0     
                self.past_rewards.append(self.R)                      
           
        elif self.issue == 2:
            
            new_state = state
            pos_live = 0
            
            if state[self.posR, self.posC] == 2:
                self.R = con.penalty
                # coil
            elif state[self.posR, self.posC] == 3:
                self.R = con.penalty - 1
                # outside air
            else:
                self.R = con.penalty
                        
            if len(self.past_actions) > 4:            
                self.repeat_count = all(elem == self.past_actions[-4] for elem in self.past_actions[-3:])
            else:
                self.repeat_count = 0
                
            if len(self.past_rewards) > 4:                
                self.neg_reward = all(elem < 0 for elem in self.past_rewards[-4:])
            else:
                self.neg_reward = 0
            
            if self.posC>20 or self.posR >9:
                pos_live = 1 
            else:
                pos_live = 0
                
            if self.repeat_count > 0 and self.neg_reward > 0 and pos_live>0:
                self.posC = self.coil_start + self.coil_width + self.worm_step_size -2
                self.posR = self.worm_step_size + 1
                self.lives -= 0.25
                new_state = self.clean_up(state)
                self.R = con.penalty * con.penalty_mul
                
            self.past_rewards.append(self.R)
            
        unique, counts = np.unique(new_state, return_counts=True)
        index_4 = np.where (unique == 4)
        index_5 = np.where (unique == 5)
        #print(index_5)
        #print(index_1)
        try:
            self.count = np.asscalar(counts[index_4[0]] + 1)            
            
        except:
            self.count = 0        
            
        self.material_left = (con.max_iron - np.copy(self.count))/con.max_iron
        self.steps_left = (con.max_steps - np.copy(self.step_count))/con.max_steps
        new_state = self.clean_up(new_state)
        
        if self.count > con.max_iron or self.step_count == self.max_steps or self.lives<0.1:
            self.done = 1.0            
        
        if new_state[self.posR, self.posC] != 5:
            raise('Problem in code. Worm outside the geometry')
            
        elif np.asscalar(counts[index_5[0]]) > 1:
            raise('Problem in code. More than one worm heads <<MEDUSA>>')
        
        self.past_states.append(np.copy(new_state))
        self.past_actions.append(np.copy(action))
#        , "Changes:", changes-22
        
        new_state2 = new_state[self.buffer: self.buffer+15, self.coil_start:27]
        new_state3 = np.dstack((new_state2, np.copy(self.B_state[:15, self.coil_start:27])))                

        self.past_BnStates.append(np.copy(new_state3))
        
#        fd = open(eps_dir + str(frame_idx) + '.csv','a')
#        writer=csv.writer(fd,delimiter=',',quoting=csv.QUOTE_MINIMAL)
#        writer.writerow([self.step_count, R, count])
#        fd.close()
#        new_state = np.expand_dims(new_state, axis= -1)    
#        state_index = np.argmax(new_state[1:16, 5:26])

        new_state4 = np.copy(new_state3.transpose((2, 0, 1)))
        
        return(new_state4, self.R, self.done, {})
#            , self.issue
#############################################################################
#                             Reset Environment                             #       
#############################################################################
        
    def reset(self, geo_dynamic= False):
        
    #    state_dim = [21, 31]  # size of geometry

        self.worm_step_size = 3  
        self.lives = 0.75
        self.material_left = (con.max_iron - 9)/con.max_iron
        self.steps_left = (con.max_steps - 1)/con.max_steps

        self.done = 0.0
        
        self.max_steps = con.max_steps
        self.past_rewards = []
        self.past_states = []
        self.past_actions = []
        self.state_hist = []
        self.past_BnStates = []
        self.past_forces = []
        self.net_force = 0.0
        self.R = 0.0
        self.count = 0                
        self.step_count = 0
        self.B_state = np.zeros([15,])
        
        if geo_dynamic == True:
            self.coil_start = np.random.randint(1, 6)        
            self.arm_start = np.random.randint(25, 31)
            self.coil_width = np.random.randint(3, 8- self.coil_start + 1)
            self.coil2_start = np.random.randint(self.coil_start + self.coil_width + 4, self.coil_start + self.coil_width + 8)  
            self.arm_width = np. random.randint(3, 35 - self.arm_start)
            self.arm_length = np.random.randint(13, 17)
            self.window_length = np.random.randint(self.arm_length, 17)
            self.coil_length = np.random.randint(6, self.window_length -5)
            self.window_width = self.arm_start - self.coil_start - self.coil_width - 1
            self.coil_turns = 500
            self.coil_current = 1

        else:               
            self.coil_start = 2
            self.arm_start = 27
            self.coil_width = 3
            self.coil2_start = 8 + 8 - (self.coil_start + self.coil_width)
            self.arm_width = 6
            self.arm_length = 15
            self.window_length = 15
            self.coil_length = 9
            self.window_width = 21
            self.coil_turns = np.random.choice(self.coil_turns_choice)
            self.coil_current = np.random.choice(self.coil_current_choice)                  

        self.create_geo()
                
        
        self.buffer = 3
        ob = np.ones(self.env_dim)*(-3)
        
        ob[self.buffer : self.buffer + self.window_length, 
           self.coil_start + self.coil_width : 
               self.coil_start + self.coil_width + self.window_width] = 0
        
        ob[self.buffer: self.buffer + self.coil_length, 
           self.coil_start: self.coil_start + self.coil_width] = -self.coil_current
        
        coil2_start = 8 + 8 - (self.coil_start + self.coil_width)
        
        ob[self.buffer: self.buffer + self.coil_length , 
           coil2_start : coil2_start + self.coil_width] = -self.coil_current
        
        ob[self.buffer: self.buffer + self.arm_length, 
           self.arm_start : self.arm_start + self.arm_width] = -4
           
        ob[self.buffer + self.coil_length:, self.coil_start+1] = np.copy(self.lives)
        ob[self.buffer + self.coil_length:, self.coil_start] = np.copy(self.material_left)
        ob[self.buffer + self.coil_length:, self.coil_start+2] = np.copy(self.steps_left)

        #print('RESETing with a state[',self.posR,',',self.posC, '] = ', ob[self.posR, self.posC])
        self.posC = self.posC = self.coil_start + self.coil_width + self.worm_step_size -2
        self.posR = self.worm_step_size + 1
                  
        self.state_hist.append((np.copy(ob[self.buffer:-1, :])))
        
        ob = self.move(ob)    
#        ob = self.clean_up(ob)
        self.past_states.append(np.copy(ob))
        self.past_actions.append(4)
        F = self.reward(ob)
        self.past_forces.append(np.copy(F))
        self.past_rewards.append(np.copy(F))
        warnings.filterwarnings("ignore")
        imageio.imwrite(FEMM_PATH + 'first.png', img_as_uint(np.abs(ob)/np.max(ob)))
        self.frame_idx += 1
        
        ob2 = ob[self.buffer: self.buffer + 15, self.coil_start:27]
        ob3 = np.dstack((ob2, np.copy(self.B_state[:15, self.coil_start:27])))
        
        self.past_BnStates.append(np.copy(ob3))        
        ob4 = np.copy(ob3.transpose((2, 0, 1)))

        return(ob4)

    def render(self, train= False):

        if train == True:    
            eps_dir = FEMM_PATH + TRAIN_PATH + '\\Eps_' + str(self.frame_idx) + '\\'
        else:
            eps_dir = FEMM_PATH + MODEL_PATH + 'Eps_' + str(self.frame_idx) + '\\'   
        
        if not (os.path.exists(FEMM_PATH + MODEL_PATH)):
            os.mkdir(FEMM_PATH + MODEL_PATH)

        if not (os.path.exists(eps_dir)):
            os.mkdir(eps_dir)

        toP = np.copy(self.past_states[-1])
        toP = toP[self.buffer : self.buffer+15, 5:26]
#        toP[toP == 1] = 5
        toP = np.abs(toP)/np.max(np.abs(toP))
        imageio.imwrite(eps_dir + str(self.frame_idx) + '_Step-' + 
                        str(self.step_count) + '_Issue-' + str(self.issue) + 
                        '_Amp-' + str(self.coil_current) + 
                        '_reward-' + str(self.R) + '_action-' + 
                        str(self.past_actions[-1]) + '_lives-' + 
                        str(self.lives)+ '_posR-'+ str(self.posR) + 
                        '_posC-'+ str(self.posC) + '.png', img_as_uint(toP))
        
def save_file(src, fileName):
    dst = FEMM_PATH + MODEL_PATH + fileName + '.py'    
    copyfile(src, dst)

def get_file(path_file):
    path_components = path_file.split('\\')
    for component in path_components:
        file_sre = re.search('(.*).py', component)
#        file_name = file_sre.group(1)
    return file_sre     

def get_file_back_slash(path_file):
    path_components = path_file.split('/')
    for component in path_components:
        file_sre = re.search('(.*).py', component)
#        file_name = file_sre.group(1)
    return file_sre     

def create_txt_file(txt, name):
    with open(FEMM_PATH + MODEL_PATH + name + "_history.txt", "w") as myfile:
        myfile.write(txt)
        
def append_txt_file(txt, name):
    with open(FEMM_PATH + MODEL_PATH + name + "_history.txt", "a") as myfile:
            myfile.write(txt)
      
def save_obj(obj, name):    
    with open(FEMM_PATH + MODEL_PATH + name + '.pkl', 'wb') as f:        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)            

def load_obj(name ):    
    with open('obj/' + name + '.pkl', 'rb') as f:        
        return pickle.load(f)            
        


In [0]:
#import gym
import logging
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kls
import tensorflow.keras.optimizers as ko

import constants as con
from env_Worm_B_1N3_B_Channel_5_33 import WormFemmEnv2, append_txt_file, create_txt_file

#import matplotlib
#import matplotlib.pyplot as plt

class ProbabilityDistribution(tf.keras.Model):
    def call(self, logits):
        # sample a random categorical action from given logits
        return tf.squeeze(tf.random.categorical(logits, 1), axis=-1)

class Model(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__('mlp_policy')
        # no tf.get_variable(), just simple Keras API
        self.conv1 = kl.Convolution2D(filters= 8, kernel_size = 3, padding= "same", activation='relu')
        self.conv2 = kl.Convolution2D(filters= 4, kernel_size = 3, padding= "same", activation='relu')
        self.flatten = kl.Flatten()
        
        self.hidden1 = kl.Dense(con.hidden_size[0], activation='relu')
        self.hidden2 = kl.Dense(con.hidden_size[1], activation='relu')
        
        self.value = kl.Dense(1, name='value')
        
        # logits are unnormalized log probabilities
        self.logits = kl.Dense(num_actions, name='policy_logits')
        self.dist = ProbabilityDistribution()

    def call(self, inputs):
        # inputs is a numpy array, convert to Tensor
        x = tf.convert_to_tensor(inputs)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        
        # separate hidden layers from the same input tensor
        hidden_logs = self.hidden1(x)
        hidden_vals = self.hidden2(x)
        
        return self.logits(hidden_logs), self.value(hidden_vals)

    def action_value(self, obs):
        # executes call() under the hood
        logits, value = self.predict(obs)
        action = self.dist.predict(logits)
        # a simpler option, will become clear later why we don't use it
        # action = tf.random.categorical(logits, 1)
        return np.squeeze(action, axis=-1), np.squeeze(value, axis=-1)
    
class A2CAgent:
    def __init__(self, model):
        # hyperparameters for loss terms, gamma is the discount coefficient
        self.params = {
            'gamma'     : con.gamma,
            'value'     : con.value_multiplier,
            'entropy'   : con.entropy
        }
        self.model = model
        self.model.compile(
            optimizer=ko.RMSprop(lr=con.lr),
            # define separate losses for policy logits and value estimate
            loss=[self._logits_loss, self._value_loss]
        )
    
    def train(self, env, batch_sz=con.max_steps*3, updates=500):
        # storage helpers for a single batch of data
        actions = np.empty((batch_sz,), dtype=np.int32)
        rewards, dones, values = np.empty((3, batch_sz))
        observations = np.empty((batch_sz,) +  tuple(con.state_size))
        # training loop: collect samples, send to optimizer, repeat updates times
        ep_rews = [0.0]
        next_obs = env.reset()
        next_obs = np.concatenate((np.copy(next_obs), np.copy(next_obs)), axis=0)

        for update in range(updates):
            for step in range(batch_sz):
                observations[step] = next_obs.copy()
                actions[step], values[step] = self.model.action_value(next_obs[None, :])
                next_obs, rewards[step], dones[step], _ = env.step(actions[step])
                
                next_obs = np.concatenate((np.copy(next_obs), np.copy(observations[-1, :2, :, :])), axis= 0)
#                env.render(train=True)
                ep_rews[-1] += rewards[step]
                if dones[step]:
                    ep_rews.append(0.0)                    
                    logging.info("Episode: %03d, Reward: %.2f, Count: %d, Net Force: %.3f" % (env.frame_idx, ep_rews[-2], env.count, env.net_force))
                    append_txt_file("\n Episode: %03d, Reward: %.2f, Count: %d, Net Force: %.3f" % (env.frame_idx, ep_rews[-2], env.count, env.net_force), file_name)
                    next_obs = env.reset()
                    next_obs = np.concatenate((np.copy(next_obs), np.copy(next_obs)), axis=0)

            _, next_value = self.model.action_value(next_obs[None, :])
            returns, advs = self._returns_advantages(rewards, dones, values, next_value)
            # a trick to input actions and advantages through same API
            acts_and_advs = np.concatenate([actions[:, None], advs[:, None]], axis=-1)
            # performs a full training step on the collected batch
            # note: no need to mess around with gradients, Keras API handles it
            losses = self.model.train_on_batch(observations, [acts_and_advs, returns])
            logging.debug("[%d/%d] Losses: %s" % (update+1, updates, losses))
            
            if update % con.playTime == 0:
                print('Playing after Epoch: {:d} and update {:d} ---> \n'.format(env.frame_idx, update))
                append_txt_file('\n \t Playing after Epoch: {:d} and update {:d} ---> \n'.format(env.frame_idx, update), file_name)
                rewards_sum, net_force, iron_c = agent.test(env)
                
                print("Total Episode Reward: {:.2f} iron_c: {} with net force {:.2f}".format(rewards_sum, iron_c, net_force))
                append_txt_file("\n \t Total Episode Reward: {:.2f} iron_c: {} with net force {:.2f}".format(rewards_sum, iron_c, net_force), file_name)
                
        return ep_rews

    def test(self, env, render=True):
        obs, done, ep_reward = env.reset(), False, 0
        obs = np.concatenate((np.copy(obs), np.copy(obs)), axis=0)

        while not done:
            action, _ = self.model.action_value(obs[None, :])
            next_obs, reward, done, _ = env.step(action)
            obs = np.concatenate((np.copy(next_obs), np.copy(obs[:2, :, :])), axis= 0)

            ep_reward += reward
            if render:
                env.render(train= False)                
        return ep_reward, env.net_force, env.count

    def _returns_advantages(self, rewards, dones, values, next_value):
        # next_value is the bootstrap value estimate of a future state (the critic)
        returns = np.append(np.zeros_like(rewards), next_value, axis=-1)
        # returns are calculated as discounted sum of future rewards
        for t in reversed(range(rewards.shape[0])):
            returns[t] = rewards[t] + self.params['gamma'] * returns[t+1] * (1-dones[t])
        returns = returns[:-1]
        # advantages are returns - baseline, value estimates in our case
        advantages = returns - values
        return returns, advantages
    
    def _value_loss(self, returns, value):
        # value loss is typically MSE between value estimates and returns
        return self.params['value']*kls.mean_squared_error(returns, value)

    def _logits_loss(self, acts_and_advs, logits):
        # a trick to input actions and advantages through same API
        actions, advantages = tf.split(acts_and_advs, 2, axis=-1)
        # sparse categorical CE loss obj that supports sample_weight arg on call()
        # from_logits argument ensures transformation into normalized probabilities
        weighted_sparse_ce = kls.SparseCategoricalCrossentropy(from_logits=True)
        # policy loss is defined by policy gradients, weighted by advantages
        # note: we only calculate the loss on the actions we've actually taken
        actions = tf.cast(actions, tf.int32)
        policy_loss = weighted_sparse_ce(actions, logits, sample_weight=advantages)
        # entropy loss can be calculated via CE over itself
        entropy_loss = kls.categorical_crossentropy(logits, logits, from_logits=True)
        # here signs are flipped because optimizer minimizes
        return policy_loss - self.params['entropy']*entropy_loss
        
env = WormFemmEnv2()
model = Model(num_actions=con.action_dim)
#model.action_value(env.reset()[None, :])  

#env = gym.make('CartPole-v0')
#model = Model(num_actions=env.action_space.n)
agent = A2CAgent(model)  

file_name = "A2C_lr_{:.4f}_gamma_{:.2f}_entropy_{:.4f}_value_{:.2f}".format(con.lr, con.gamma, con.entropy, con.value_multiplier)
text = 'A2C_Tf2_Stack_5_33'
create_txt_file(text, file_name)

#rewards_sum = agent.test(env)
#print("Total Episode Reward: %d out of 200" % agent.test(env))

logging.getLogger().setLevel(logging.INFO)

rewards_history = agent.train(env)
print("Finished training.")

